In [160]:
import requests
from datetime import datetime, timedelta

class UserEventsRepo(object):
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({'PRIVATE-TOKEN': '#_YOUR_TOKEN_HERE'})

    def get_all_users(self, active=True):
        response = self.session.get('http://gitlab.coinecta.com/api/v4/users?active=true')
        return response.json()

    def get_events(self, user_ids, num_of_days=7):
        contribs = []
        for user_id in user_ids:
            for i in range(0, num_of_days):
                before = (datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d')
                after = (datetime.today() - timedelta(days=i+2)).strftime('%Y-%m-%d')
                response = self.session.get('http://gitlab.coinecta.com/api/v4/users/%s/events?before=%s&after=%s&per_page=5000' % (user_id, before, after))
                user_contrib = response.json()
                contribs.append(user_contrib)
        return [item for sublist in contribs for item in sublist]

In [165]:
from datetime import datetime, timedelta

users_repo = UserEventsRepo()
users = users_repo.get_all_users()
uids = [d['id'] for d in users]
#after = (datetime.today() - timedelta(days=3)).strftime('%Y-%m-%d')
events = users_repo.get_events(uids, 5)

In [ ]:
import math

eventsdf = pd.DataFrame(events)
eventsdf['created_at'] = eventsdf['created_at'].apply(lambda dt: datetime.strptime(dt[:-5], '%Y-%m-%dT%H:%M:%S').date())
events_grouped = eventsdf.groupby(['created_at', 'author_username'])['action_name'].count()
heat_data = events_grouped.to_frame().reset_index()
heat_data['action_name'] = heat_data['action_name'].apply(lambda act: math.log(int(act)))
heat_data['created_at'] = heat_data['created_at'].astype(str)
#heat_data
heat_data = heat_data.pivot('author_username', 'created_at', 'action_name').fillna(0)
#heat_data
ax = sns.heatmap(heat_data, cmap="viridis")